In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-for-financial-news/all-data.csv
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_66Agree.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_AllAgree.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/README.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/License.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_75Agree.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_50Agree.txt


In [55]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
from xgboost import XGBClassifier

nlp = spacy.load("en_core_web_sm")

In [16]:
# Load the dataset
data = pd.read_csv('/kaggle/input/sentiment-analysis-for-financial-news/all-data.csv',names=["sentiment", "text"],encoding="utf-8", encoding_errors="replace")

In [17]:
data.head()

,sentiment,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [18]:
data.describe()

,sentiment,text
count,4846,4846
unique,3,4838
top,neutral,TELECOMWORLDWIRE-7 April 2006-TJ Group Plc sel...
freq,2879,2


In [19]:
data.sentiment.value_counts()

sentiment
neutral     2879
positive    1363
negative     604
Name: count, dtype: int64

In [27]:
label_encoder = LabelEncoder()
data['sentiment_encoded'] = label_encoder.fit_transform(data['sentiment'])

In [31]:
data.head()

,sentiment,text,sentiment_encoded
0,neutral,"According to Gran , the company has no plans t...",1
1,neutral,Technopolis plans to develop in stages an area...,1
2,negative,The international electronic industry company ...,0
3,positive,With the new production plant the company woul...,2
4,positive,According to the company 's updated strategy f...,2


In [43]:
def clean_text(text):
    # Process the text using spaCy
    doc = nlp(text)
    
    # Lemmatization and stop word removal
    cleaned_tokens = [token.lemma_ for token in doc if not token.is_stop]
    
    # Join the cleaned tokens back into a string
    cleaned_text = ' '.join(cleaned_tokens)
    
    return cleaned_text

In [44]:
data['cleaned_text'] = data['text'].apply(clean_text)

In [45]:
data

,sentiment,text,sentiment_encoded,cleaned_text
0,neutral,"According to Gran , the company has no plans t...",1,"accord Gran , company plan production Russia ,..."
1,neutral,Technopolis plans to develop in stages an area...,1,"technopoli plan develop stage area 100,000 squ..."
2,negative,The international electronic industry company ...,0,international electronic industry company Elco...
3,positive,With the new production plant the company woul...,2,new production plant company increase capacity...
4,positive,According to the company 's updated strategy f...,2,accord company update strategy year 2009 - 201...
...,...,...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...,0,LONDON MarketWatch -- share price end lower Lo...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,1,Rinkuskiai beer sale fall 6.5 cent 4.16 millio...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...,0,operate profit fall EUR 35.4 mn EUR 68.8 mn 20...
4844,negative,Net sales of the Paper segment decreased to EU...,0,net sale Paper segment decrease EUR 221.6 mn s...


In [52]:
# TF-IDF feature extraction
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(data['cleaned_text'])

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, data['sentiment_encoded'], test_size=0.2, random_state=42)

In [56]:
# Initialize and train an XGBoost classifier

xgb_classifier = XGBClassifier(objective='multi:softprob', random_state=0)
xgb_classifier.fit(X_train, y_train)

# Make predictions
y_pred_xgb = xgb_classifier.predict(X_test)

# Model evaluation
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f'XGBoost Accuracy: {accuracy_xgb:.2f}')

# More detailed metrics
print(classification_report(y_test, y_pred_xgb))

XGBoost Accuracy: 0.76
              precision    recall  f1-score   support

           0       0.76      0.54      0.63       110
           1       0.77      0.90      0.83       571
           2       0.73      0.56      0.64       289

    accuracy                           0.76       970
   macro avg       0.75      0.67      0.70       970
weighted avg       0.76      0.76      0.75       970



In [58]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=0)
rf_classifier.fit(X_train, y_train)

# Make predictions
y_pred_rgb = rf_classifier.predict(X_test)

accuracy_rgb = accuracy_score(y_test, y_pred_rgb)
print(f'Random Forest Accuracy: {accuracy_rgb:.2f}')

# More detailed metrics
print(classification_report(y_test, y_pred_rgb))

Random Forest Accuracy: 0.76
              precision    recall  f1-score   support

           0       0.81      0.42      0.55       110
           1       0.75      0.95      0.84       571
           2       0.77      0.50      0.61       289

    accuracy                           0.76       970
   macro avg       0.78      0.62      0.66       970
weighted avg       0.76      0.76      0.74       970



## Lets Hypertune the Model

In [59]:
import optuna
import xgboost as xgb
from sklearn.model_selection import cross_val_score

In [60]:
def objective(trial):
    data_dmatrix = xgb.DMatrix(data=X_train, label=y_train)

    params = {
        'objective': 'multi:softmax',  # Use 'multi:softmax' for multi-class classification
        'num_class': 3,  # Number of classes (e.g., 'up', 'down', 'neutral')
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
    }

    model = xgb.XGBClassifier(**params)

    # Use cross-validation to evaluate the model
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy')

    return 1.0 - scores.mean()  # Optuna minimizes the objective function, so we negate accuracy


In [62]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=25)  # You can adjust the number of trials


[I 2023-10-25 01:52:43,736] A new study created in memory with name: no-name-9d6f76d1-7a44-4c6f-a32e-d3146a6d2494
/tmp/ipykernel_32/1319473264.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
[I 2023-10-25 01:52:53,805] Trial 0 finished with value: 0.2776057791537667 and parameters: {'max_depth': 3, 'learning_rate': 0.028169115383714416, 'n_estimators': 270}. Best is trial 0 with value: 0.2776057791537667.
/tmp/ipykernel_32/1319473264.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
[I 2023-10-25 01:53:08,484] Trial 1 finish

In [63]:
best_params = study.best_params

# Train the final model with the best hyperparameters
final_model = xgb.XGBClassifier(**best_params)
final_model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.08505859117069119,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=660, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [64]:
y_pred_final = final_model.predict(X_test)

accuracy_final = accuracy_score(y_test, y_pred_final)
print(f'Random Forest Accuracy: {accuracy_final:.2f}')

# More detailed metrics
print(classification_report(y_test, y_pred_final))

Random Forest Accuracy: 0.76
              precision    recall  f1-score   support

           0       0.76      0.53      0.62       110
           1       0.77      0.92      0.84       571
           2       0.76      0.55      0.64       289

    accuracy                           0.76       970
   macro avg       0.76      0.67      0.70       970
weighted avg       0.76      0.76      0.75       970

